<a href="https://colab.research.google.com/github/LiibanMo/ML-Algorithms/blob/main/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [312]:
import numpy as np
import pandas as pd

In [313]:
import pandas as pd
import numpy as np
x_values = pd.DataFrame({
    'height': [6, 5.92, 5.58, 5.92, 5, 5.5, 5.42, 5.75],
    'weight': [180,190,170,165,100,150,130,150],
    'foot': [12,11,12,10,6,8,7,9]
})
# "male" to 0, and "female" to 1.
y_values = pd.Series([0,0,0,0,1,1,1,1])

sample = pd.DataFrame({
    'height': [6],
    'weight': [130],
    'foot': [8]
})

x_test = np.array([[6, 130, 8],
                  [6, 180, 10],
                   [5, 180, 7],])

In [325]:
class GaussianNaiveBayesClassifier:
    def fit(self, X, y):
        def obtain_priors(y):
            priors = []
            for class_ in np.unique(y):
                prior = len(y[y==class_]) / len(y)
                priors.append(prior)
            return priors

        self.priors = obtain_priors(y)

        def obtain_data_given_class(X, y):
            separated_data = []
            for class_ in np.unique(y):
                separated_data.append(X[y==class_])
            return separated_data

        separated_data = obtain_data_given_class(X, y)

        def obtain_statistics_for_each_feature_given_class(separated_data):
            statistics = []
            for i, data in enumerate(separated_data):
                means = data.mean(axis=0)
                vars = data.var(axis=0, ddof=1)
                stat = np.column_stack((means, vars))
                statistics.append(stat)
            return statistics

        self.statistics = obtain_statistics_for_each_feature_given_class(separated_data)

        return self

    def predict(self, X):
        def obtain_likelihoods(x, statistics):
            def gaussian(x, mean, var):
                return 1 / np.sqrt(2 * np.pi * var) * np.exp(-(x - mean)**2 / (2 * var))

            likelihoods = []
            for idx in range(len(statistics)):
                likelihood = 1
                for obs, (mean, var) in zip(x, statistics[idx]):
                    likelihood *= gaussian(obs, mean, var)
                likelihoods.append(likelihood)

            return likelihoods

        def obtain_posteriors(priors, likelihoods):
            posteriors = []
            for prior, likelihood in zip(priors, likelihoods):
                posteriors.append(prior * likelihood)
            return posteriors

        posteriors = []
        for x in X:
            likelihoods = obtain_likelihoods(x, self.statistics)
            posteriors.append(obtain_posteriors(self.priors, likelihoods))

        return np.asarray(posteriors).argmax(axis=1)


In [326]:
model = GaussianNaiveBayesClassifier()
fitted_model = model.fit(X, y)
y_pred = fitted_model.predict(x_test)
y_pred

array([1, 0, 1])